# Day9_1: 가설 검정 (Hypothesis Testing)

## 📚 학습 목표

**Part 1: 기초 개념**
1. 귀무가설과 대립가설의 차이 이해하기
2. p-value의 의미와 해석 방법 배우기
3. 유의수준(α)과 신뢰수준의 관계 파악하기
4. Type I 오류와 Type II 오류 구분하기
5. 가설 검정 프로세스 전체 흐름 이해하기

**Part 2: 검정 방법**
1. t-test (단일, 독립, 대응) 구분하여 사용하기
2. chi-square 검정으로 범주형 데이터 분석하기
3. ANOVA로 3개 이상 그룹 비교하기
4. 상관계수(Pearson, Spearman) 계산하고 해석하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 가설 검정 | 데이터 기반 의사결정 | "신규 UI가 전환율을 높였나?" |
| t-test | A/B 테스트 분석 | 캠페인 A vs B 성과 비교 |
| chi-square | 설문조사 분석 | 연령대별 제품 선호도 차이 검증 |
| ANOVA | 다중 그룹 비교 | 지역별 매출 차이 분석 |
| 상관계수 | 변수 간 관계 파악 | 광고비와 매출의 상관성 |

**분석가 관점**: 가설 검정은 "이 차이가 우연인가, 실제 효과인가?"를 통계적으로 판단하는 핵심 도구입니다!

---

# Part 1: 기초 개념

---

## 1.1 가설 검정의 기본 개념

### 귀무가설(H₀) vs 대립가설(H₁)

- **귀무가설(Null Hypothesis, H₀)**: "차이가 없다" 또는 "효과가 없다"는 가정
- **대립가설(Alternative Hypothesis, H₁)**: "차이가 있다" 또는 "효과가 있다"는 주장

가설 검정의 목표는 **데이터를 통해 귀무가설을 기각할 충분한 증거가 있는지 판단**하는 것입니다.

In [1]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


### 💡 실무 예시: A/B 테스트 시나리오

**상황**: 이커머스 사이트에서 새로운 결제 버튼 디자인을 테스트합니다.

- **H₀ (귀무가설)**: 기존 버튼과 신규 버튼의 전환율에 차이가 없다
- **H₁ (대립가설)**: 기존 버튼과 신규 버튼의 전환율에 차이가 있다

In [2]:
# A/B 테스트 데이터 (전환율 %)
group_a = [3.2, 3.5, 3.1, 3.8, 3.4, 3.6, 3.3, 3.7, 3.2, 3.5]  # 기존 버튼
group_b = [4.1, 4.3, 4.0, 4.5, 4.2, 4.4, 4.1, 4.6, 4.3, 4.2]  # 신규 버튼

print(f"A그룹 평균 전환율: {np.mean(group_a):.2f}%")
print(f"B그룹 평균 전환율: {np.mean(group_b):.2f}%")
print(f"차이: {np.mean(group_b) - np.mean(group_a):.2f}%p")

A그룹 평균 전환율: 3.43%
B그룹 평균 전환율: 4.27%
차이: 0.84%p


---

## 1.2 p-value의 의미

### p-value란?

**p-value**: 귀무가설이 참일 때, 현재 관측된 데이터(또는 더 극단적인 데이터)가 나올 확률

- **p-value가 작을수록** → 귀무가설이 참일 가능성이 낮음 → 귀무가설 기각
- **p-value가 클수록** → 귀무가설이 참일 가능성이 높음 → 귀무가설 채택

### 해석 기준

일반적으로 **유의수준(α) = 0.05**를 기준으로 사용합니다:

- **p < 0.05**: 통계적으로 유의함 → 귀무가설 기각
- **p ≥ 0.05**: 통계적으로 유의하지 않음 → 귀무가설 채택

In [3]:
# p-value 해석 예시
p_values = [0.001, 0.03, 0.08, 0.15, 0.50]
alpha = 0.05

for p in p_values:
    if p < alpha:
        result = "✅ 귀무가설 기각 (통계적으로 유의함)"
    else:
        result = "❌ 귀무가설 채택 (통계적으로 유의하지 않음)"
    print(f"p-value = {p:.3f} → {result}")

p-value = 0.001 → ✅ 귀무가설 기각 (통계적으로 유의함)
p-value = 0.030 → ✅ 귀무가설 기각 (통계적으로 유의함)
p-value = 0.080 → ❌ 귀무가설 채택 (통계적으로 유의하지 않음)
p-value = 0.150 → ❌ 귀무가설 채택 (통계적으로 유의하지 않음)
p-value = 0.500 → ❌ 귀무가설 채택 (통계적으로 유의하지 않음)


### 💡 실무 예시: 마케팅 캠페인 효과 검증

**상황**: 이메일 마케팅 캠페인 후 매출이 증가했는지 확인합니다.

In [4]:
# 캠페인 전후 일별 매출 (만원)
before_campaign = [120, 135, 128, 142, 130, 125, 138, 132, 127, 140]
after_campaign = [145, 158, 152, 165, 148, 155, 162, 150, 157, 160]

# t-test 수행
t_stat, p_value = stats.ttest_ind(before_campaign, after_campaign)

print(f"캠페인 전 평균 매출: {np.mean(before_campaign):.1f}만원")
print(f"캠페인 후 평균 매출: {np.mean(after_campaign):.1f}만원")
print(f"\nt-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("\n✅ 결론: 캠페인이 매출 증가에 통계적으로 유의한 효과가 있습니다!")
else:
    print("\n❌ 결론: 캠페인 효과가 통계적으로 유의하지 않습니다.")

캠페인 전 평균 매출: 131.7만원
캠페인 후 평균 매출: 155.2만원

t-statistic: -7.8059
p-value: 0.0000

✅ 결론: 캠페인이 매출 증가에 통계적으로 유의한 효과가 있습니다!


---

## 1.3 유의수준(α)과 신뢰수준

### 유의수준(Significance Level, α)

- **유의수준(α)**: 귀무가설이 참인데도 기각할 최대 허용 확률
- 일반적으로 **α = 0.05 (5%)** 또는 **α = 0.01 (1%)** 사용

### 신뢰수준(Confidence Level)

- **신뢰수준 = 1 - α**
- α = 0.05 → 신뢰수준 95%
- α = 0.01 → 신뢰수준 99%

In [5]:
# 다양한 유의수준 비교
significance_levels = pd.DataFrame({
    '유의수준(α)': [0.10, 0.05, 0.01],
    '신뢰수준': ['90%', '95%', '99%'],
    '엄격도': ['낮음', '보통', '높음'],
    '활용 예시': [
        '탐색적 분석',
        '일반적인 A/B 테스트',
        '의료/금융 의사결정'
    ]
})

print(significance_levels.to_string(index=False))

 유의수준(α) 신뢰수준 엄격도        활용 예시
    0.10  90%  낮음       탐색적 분석
    0.05  95%  보통 일반적인 A/B 테스트
    0.01  99%  높음   의료/금융 의사결정


### 💡 실무 예시: 신약 임상시험 vs A/B 테스트

- **신약 임상시험**: α = 0.01 (99% 신뢰수준) → 매우 엄격한 기준
- **웹사이트 A/B 테스트**: α = 0.05 (95% 신뢰수준) → 일반적인 기준
- **탐색적 분석**: α = 0.10 (90% 신뢰수준) → 덜 엄격한 기준

---

## 1.4 Type I 오류와 Type II 오류

### 오류의 종류

| 실제 상황 \ 결정 | H₀ 기각 | H₀ 채택 |
|-----------------|---------|--------|
| **H₀가 참** | ❌ Type I 오류 (α) | ✅ 올바른 결정 |
| **H₁이 참** | ✅ 올바른 결정 | ❌ Type II 오류 (β) |

- **Type I 오류 (α)**: 실제로는 차이가 없는데 "차이가 있다"고 잘못 판단 (False Positive)
- **Type II 오류 (β)**: 실제로는 차이가 있는데 "차이가 없다"고 잘못 판단 (False Negative)

In [6]:
# 오류 유형 시각화 테이블
error_examples = pd.DataFrame({
    '오류 유형': ['Type I 오류', 'Type II 오류'],
    '확률 기호': ['α (알파)', 'β (베타)'],
    '일반 용어': ['False Positive', 'False Negative'],
    '의료 검사 예시': [
        '건강한데 "병 있다" 진단',
        '병 있는데 "건강하다" 진단'
    ],
    '마케팅 예시': [
        '효과 없는데 "효과 있다" 판단',
        '효과 있는데 "효과 없다" 판단'
    ]
})

print(error_examples.to_string(index=False))

     오류 유형  확률 기호          일반 용어        의료 검사 예시            마케팅 예시
 Type I 오류 α (알파) False Positive  건강한데 "병 있다" 진단 효과 없는데 "효과 있다" 판단
Type II 오류 β (베타) False Negative 병 있는데 "건강하다" 진단 효과 있는데 "효과 없다" 판단


### 💡 실무 예시: 스팸 필터링

- **Type I 오류**: 정상 메일을 스팸으로 잘못 분류 → 중요한 메일 놓침
- **Type II 오류**: 스팸 메일을 정상으로 잘못 분류 → 스팸이 받은편지함에 도착

어떤 오류가 더 심각한지는 **비즈니스 컨텍스트**에 따라 다릅니다!

---

## 1.5 가설 검정 프로세스

### 5단계 워크플로우

1. **가설 설정**: H₀와 H₁ 정의
2. **유의수준 결정**: α 설정 (보통 0.05)
3. **검정 통계량 선택**: t-test, chi-square, ANOVA 등
4. **검정 수행**: p-value 계산
5. **결론 도출**: p-value와 α 비교하여 H₀기각 여부 결정

In [7]:
# 가설 검정 프로세스 예시: 고객 만족도 개선 검증

# Step 1: 가설 설정
print("=" * 60)
print("Step 1: 가설 설정")
print("=" * 60)
print("H₀: 신규 서비스 전후 고객 만족도에 차이가 없다")
print("H₁: 신규 서비스 후 고객 만족도가 증가했다\n")

# Step 2: 유의수준 결정
alpha = 0.05
print(f"Step 2: 유의수준 α = {alpha}\n")

# Step 3: 데이터 수집
satisfaction_before = [3.2, 3.5, 3.8, 3.1, 3.6, 3.4, 3.7, 3.3, 3.5, 3.4]
satisfaction_after = [4.1, 4.3, 4.5, 3.9, 4.2, 4.0, 4.4, 4.1, 4.3, 4.2]

print(f"Step 3: 데이터 수집")
print(f"서비스 전 평균: {np.mean(satisfaction_before):.2f}")
print(f"서비스 후 평균: {np.mean(satisfaction_after):.2f}\n")

# Step 4: 검정 수행 (대응 표본 t-test)
t_stat, p_value = stats.ttest_rel(satisfaction_before, satisfaction_after)
print(f"Step 4: 검정 수행 (대응 표본 t-test)")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}\n")

# Step 5: 결론 도출
print("=" * 60)
print("Step 5: 결론 도출")
print("=" * 60)
if p_value < alpha:
    print(f"✅ p-value ({p_value:.4f}) < α ({alpha})")
    print("→ 귀무가설 기각")
    print("→ 결론: 신규 서비스가 고객 만족도를 유의하게 향상시켰습니다!")
else:
    print(f"❌ p-value ({p_value:.4f}) ≥ α ({alpha})")
    print("→ 귀무가설 채택")
    print("→ 결론: 신규 서비스 효과가 통계적으로 유의하지 않습니다.")

Step 1: 가설 설정
H₀: 신규 서비스 전후 고객 만족도에 차이가 없다
H₁: 신규 서비스 후 고객 만족도가 증가했다

Step 2: 유의수준 α = 0.05

Step 3: 데이터 수집
서비스 전 평균: 3.45
서비스 후 평균: 4.20

Step 4: 검정 수행 (대응 표본 t-test)
t-statistic: -24.4047
p-value: 0.0000

Step 5: 결론 도출
✅ p-value (0.0000) < α (0.05)
→ 귀무가설 기각
→ 결론: 신규 서비스가 고객 만족도를 유의하게 향상시켰습니다!


---

# Part 2: 검정 방법

---

## 2.1 t-test (t-검정)

### t-test란?

**평균의 차이를 검정**하는 방법으로, 3가지 유형이 있습니다:

1. **단일 표본 t-test**: 한 그룹의 평균이 특정 값과 다른지 검정
2. **독립 표본 t-test**: 두 독립된 그룹의 평균 차이 검정
3. **대응 표본 t-test**: 같은 대상의 전후 비교

### 1) 단일 표본 t-test (One-Sample t-test)

In [8]:
# 예시: 전국 평균 배송시간이 3일인지 검증
# H₀: 우리 회사 평균 배송시간 = 3일
# H₁: 우리 회사 평균 배송시간 ≠ 3일

delivery_days = [2.8, 3.2, 2.9, 3.5, 3.1, 2.7, 3.3, 3.0, 2.9, 3.2]
national_avg = 3.0

# 단일 표본 t-test
t_stat, p_value = stats.ttest_1samp(delivery_days, national_avg)

print(f"우리 회사 평균 배송시간: {np.mean(delivery_days):.2f}일")
print(f"전국 평균: {national_avg}일")
print(f"\nt-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("\n✅ 우리 회사 배송시간은 전국 평균과 유의한 차이가 있습니다.")
else:
    print("\n❌ 우리 회사 배송시간은 전국 평균과 유의한 차이가 없습니다.")

우리 회사 평균 배송시간: 3.06일
전국 평균: 3.0일

t-statistic: 0.7717
p-value: 0.4600

❌ 우리 회사 배송시간은 전국 평균과 유의한 차이가 없습니다.


### 2) 독립 표본 t-test (Independent Two-Sample t-test)

In [9]:
# 예시: 남성과 여성의 평균 구매액 비교
# H₀: 남성 평균 구매액 = 여성 평균 구매액
# H₁: 남성 평균 구매액 ≠ 여성 평균 구매액

male_purchase = [120, 135, 150, 128, 142, 138, 145, 132, 140, 148]
female_purchase = [145, 158, 162, 148, 155, 152, 165, 150, 160, 157]

# 독립 표본 t-test
t_stat, p_value = stats.ttest_ind(male_purchase, female_purchase)

print(f"남성 평균 구매액: {np.mean(male_purchase):.1f}만원")
print(f"여성 평균 구매액: {np.mean(female_purchase):.1f}만원")
print(f"차이: {np.mean(female_purchase) - np.mean(male_purchase):.1f}만원")
print(f"\nt-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("\n✅ 남성과 여성의 구매액에 유의한 차이가 있습니다.")
else:
    print("\n❌ 남성과 여성의 구매액에 유의한 차이가 없습니다.")

남성 평균 구매액: 137.8만원
여성 평균 구매액: 155.2만원
차이: 17.4만원

t-statistic: -4.8651
p-value: 0.0001

✅ 남성과 여성의 구매액에 유의한 차이가 있습니다.


### 3) 대응 표본 t-test (Paired t-test)

In [10]:
# 예시: 다이어트 프로그램 전후 체중 비교
# H₀: 다이어트 전후 체중에 차이가 없다
# H₁: 다이어트 후 체중이 감소했다

weight_before = [75, 82, 68, 90, 78, 85, 72, 88, 80, 76]
weight_after = [72, 79, 66, 87, 75, 82, 70, 85, 78, 74]

# 대응 표본 t-test
t_stat, p_value = stats.ttest_rel(weight_before, weight_after)

# 개인별 체중 감소량
weight_loss = [before - after for before, after in zip(weight_before, weight_after)]

print(f"평균 체중 감소: {np.mean(weight_loss):.2f}kg")
print(f"\nt-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("\n✅ 다이어트 프로그램이 체중 감소에 유의한 효과가 있습니다!")
else:
    print("\n❌ 다이어트 프로그램 효과가 통계적으로 유의하지 않습니다.")

평균 체중 감소: 2.60kg

t-statistic: 15.9217
p-value: 0.0000

✅ 다이어트 프로그램이 체중 감소에 유의한 효과가 있습니다!


### 💡 실무 예시: A/B 테스트 완전 분석

In [11]:
# 랜딩페이지 A/B 테스트 - 일일 전환 수
landing_a = [45, 52, 48, 55, 50, 47, 53, 49, 51, 46]  # 기존 디자인
landing_b = [58, 62, 60, 65, 59, 63, 61, 64, 58, 62]  # 신규 디자인

# t-test 수행
t_stat, p_value = stats.ttest_ind(landing_a, landing_b)

# 결과 리포트
print("=" * 60)
print("A/B 테스트 결과 리포트")
print("=" * 60)
print(f"\n랜딩페이지 A (기존):")
print(f"  평균: {np.mean(landing_a):.1f}건/일")
print(f"  표준편차: {np.std(landing_a, ddof=1):.2f}")

print(f"\n랜딩페이지 B (신규):")
print(f"  평균: {np.mean(landing_b):.1f}건/일")
print(f"  표준편차: {np.std(landing_b, ddof=1):.2f}")

improvement = (np.mean(landing_b) - np.mean(landing_a)) / np.mean(landing_a) * 100
print(f"\n개선율: {improvement:.1f}%")

print(f"\n통계 검정:")
print(f"  t-statistic: {t_stat:.4f}")
print(f"  p-value: {p_value:.4f}")

print("\n" + "=" * 60)
if p_value < 0.05:
    print("✅ 결론: 신규 디자인이 통계적으로 유의하게 전환율을 개선했습니다!")
    print(f"   권장: 신규 디자인을 전체 사용자에게 적용하세요.")
else:
    print("❌ 결론: 두 디자인 간 유의한 차이가 없습니다.")
    print(f"   권장: 추가 테스트를 진행하거나 다른 요소를 개선하세요.")
print("=" * 60)

A/B 테스트 결과 리포트

랜딩페이지 A (기존):
  평균: 49.6건/일
  표준편차: 3.20

랜딩페이지 B (신규):
  평균: 61.2건/일
  표준편차: 2.44

개선율: 23.4%

통계 검정:
  t-statistic: -9.1076
  p-value: 0.0000

✅ 결론: 신규 디자인이 통계적으로 유의하게 전환율을 개선했습니다!
   권장: 신규 디자인을 전체 사용자에게 적용하세요.


---

## 2.2 Chi-Square 검정 (카이제곱 검정)

### Chi-Square 검정이란?

**범주형 변수 간의 관련성**을 검정하는 방법입니다.

- **H₀**: 두 범주형 변수는 독립적이다 (관련이 없다)
- **H₁**: 두 범주형 변수는 관련이 있다

### 활용 예시
- 성별과 제품 선호도의 관계
- 연령대와 구매 여부의 관계
- 지역과 정치 성향의 관계

In [12]:
# 예시: 성별과 제품 선호도 관계 분석
# H₀: 성별과 제품 선호도는 독립적이다
# H₁: 성별과 제품 선호도는 관련이 있다

# 교차표 (Contingency Table)
# 행: 성별, 열: 제품 선호도
observed = np.array([
    [30, 20, 10],  # 남성: A제품, B제품, C제품
    [15, 35, 25]   # 여성: A제품, B제품, C제품
])

# Chi-square 검정
chi2, p_value, dof, expected = stats.chi2_contingency(observed)

# 결과 출력
print("관측 빈도:")
print(pd.DataFrame(observed, 
                   index=['남성', '여성'],
                   columns=['A제품', 'B제품', 'C제품']))

print("\n기대 빈도:")
print(pd.DataFrame(expected, 
                   index=['남성', '여성'],
                   columns=['A제품', 'B제품', 'C제품']).round(2))

print(f"\nChi-square 통계량: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"자유도: {dof}")

if p_value < 0.05:
    print("\n✅ 성별과 제품 선호도는 유의한 관련이 있습니다!")
else:
    print("\n❌ 성별과 제품 선호도는 독립적입니다.")

관측 빈도:
    A제품  B제품  C제품
남성   30   20   10
여성   15   35   25

기대 빈도:
     A제품    B제품    C제품
남성  20.0  24.44  15.56
여성  25.0  30.56  19.44

Chi-square 통계량: 14.0260
p-value: 0.0009
자유도: 2

✅ 성별과 제품 선호도는 유의한 관련이 있습니다!


### 💡 실무 예시: 마케팅 채널별 전환율 비교

In [13]:
# 채널별 전환 여부
# H₀: 마케팅 채널과 전환 여부는 독립적이다
# H₁: 마케팅 채널에 따라 전환율이 다르다

# 교차표: 행(채널), 열(전환 여부)
channel_conversion = np.array([
    [150, 850],  # 이메일: 전환O, 전환X
    [200, 800],  # SNS: 전환O, 전환X
    [100, 900]   # 검색광고: 전환O, 전환X
])

# Chi-square 검정
chi2, p_value, dof, expected = stats.chi2_contingency(channel_conversion)

# 채널별 전환율 계산
channels = ['이메일', 'SNS', '검색광고']
conversion_rates = []
for i, channel in enumerate(channels):
    total = channel_conversion[i].sum()
    converted = channel_conversion[i][0]
    rate = converted / total * 100
    conversion_rates.append(rate)
    print(f"{channel}: {converted}/{total} = {rate:.1f}%")

print(f"\nChi-square 통계량: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("\n✅ 채널에 따라 전환율에 유의한 차이가 있습니다!")
    best_channel = channels[np.argmax(conversion_rates)]
    print(f"   최고 성과 채널: {best_channel} ({max(conversion_rates):.1f}%)")
else:
    print("\n❌ 채널 간 전환율에 유의한 차이가 없습니다.")

이메일: 150/1000 = 15.0%
SNS: 200/1000 = 20.0%
검색광고: 100/1000 = 10.0%

Chi-square 통계량: 39.2157
p-value: 0.0000

✅ 채널에 따라 전환율에 유의한 차이가 있습니다!
   최고 성과 채널: SNS (20.0%)


---

## 2.3 ANOVA (분산분석)

### ANOVA란?

**3개 이상의 그룹 평균을 동시에 비교**하는 방법입니다.

- **H₀**: 모든 그룹의 평균이 같다
- **H₁**: 적어도 하나의 그룹 평균이 다르다

### t-test vs ANOVA

- **t-test**: 2개 그룹 비교
- **ANOVA**: 3개 이상 그룹 비교

In [14]:
# 예시: 지역별 매출 차이 분석
# H₀: 서울, 부산, 대구의 평균 매출이 같다
# H₁: 적어도 한 지역의 매출이 다르다

seoul_sales = [120, 135, 128, 142, 130, 125, 138, 132]
busan_sales = [145, 158, 152, 165, 148, 155, 162, 150]
daegu_sales = [115, 108, 122, 118, 112, 125, 120, 116]

# One-Way ANOVA
f_stat, p_value = stats.f_oneway(seoul_sales, busan_sales, daegu_sales)

print("지역별 평균 매출:")
print(f"  서울: {np.mean(seoul_sales):.1f}만원")
print(f"  부산: {np.mean(busan_sales):.1f}만원")
print(f"  대구: {np.mean(daegu_sales):.1f}만원")

print(f"\nF-statistic: {f_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("\n✅ 지역 간 매출에 유의한 차이가 있습니다!")
    sales_dict = {'서울': np.mean(seoul_sales), 
                  '부산': np.mean(busan_sales), 
                  '대구': np.mean(daegu_sales)}
    best_region = max(sales_dict, key=sales_dict.get)
    print(f"   최고 매출 지역: {best_region} ({sales_dict[best_region]:.1f}만원)")
else:
    print("\n❌ 지역 간 매출에 유의한 차이가 없습니다.")

지역별 평균 매출:
  서울: 131.2만원
  부산: 154.4만원
  대구: 117.0만원

F-statistic: 66.3122
p-value: 0.0000

✅ 지역 간 매출에 유의한 차이가 있습니다!
   최고 매출 지역: 부산 (154.4만원)


### 💡 실무 예시: 4개 마케팅 캠페인 효과 비교

In [15]:
# 캠페인별 일일 전환 수
campaign_a = [45, 52, 48, 55, 50, 47, 53]
campaign_b = [58, 62, 60, 65, 59, 63, 61]
campaign_c = [42, 38, 45, 40, 43, 39, 44]
campaign_d = [52, 55, 53, 58, 54, 56, 57]

# ANOVA 검정
f_stat, p_value = stats.f_oneway(campaign_a, campaign_b, campaign_c, campaign_d)

# 결과 리포트
print("=" * 60)
print("마케팅 캠페인 비교 분석")
print("=" * 60)

campaigns = {
    'Campaign A (이메일)': campaign_a,
    'Campaign B (SNS)': campaign_b,
    'Campaign C (배너광고)': campaign_c,
    'Campaign D (검색광고)': campaign_d
}

for name, data in campaigns.items():
    print(f"\n{name}")
    print(f"  평균: {np.mean(data):.1f}건/일")
    print(f"  표준편차: {np.std(data, ddof=1):.2f}")

print(f"\n통계 검정:")
print(f"  F-statistic: {f_stat:.4f}")
print(f"  p-value: {p_value:.6f}")

print("\n" + "=" * 60)
if p_value < 0.05:
    print("✅ 캠페인 간 성과에 유의한 차이가 있습니다!")
    best = max(campaigns.items(), key=lambda x: np.mean(x[1]))
    print(f"   최고 성과: {best[0]} (평균 {np.mean(best[1]):.1f}건/일)")
    print(f"   권장: {best[0]} 캠페인에 예산을 집중하세요.")
else:
    print("❌ 캠페인 간 성과에 유의한 차이가 없습니다.")
    print(f"   권장: 다른 성과 지표를 분석하거나 캠페인을 재설계하세요.")
print("=" * 60)

마케팅 캠페인 비교 분석

Campaign A (이메일)
  평균: 50.0건/일
  표준편차: 3.56

Campaign B (SNS)
  평균: 61.1건/일
  표준편차: 2.41

Campaign C (배너광고)
  평균: 41.6건/일
  표준편차: 2.64

Campaign D (검색광고)
  평균: 55.0건/일
  표준편차: 2.16

통계 검정:
  F-statistic: 63.6772
  p-value: 0.000000

✅ 캠페인 간 성과에 유의한 차이가 있습니다!
   최고 성과: Campaign B (SNS) (평균 61.1건/일)
   권장: Campaign B (SNS) 캠페인에 예산을 집중하세요.


---

## 2.4 상관계수 (Correlation Coefficient)

### 상관계수란?

**두 연속형 변수 간의 선형 관계 강도**를 측정합니다.

- **값 범위**: -1 ~ +1
- **+1에 가까울수록**: 강한 양의 상관관계 (한 변수가 증가하면 다른 변수도 증가)
- **-1에 가까울수록**: 강한 음의 상관관계 (한 변수가 증가하면 다른 변수는 감소)
- **0에 가까울수록**: 상관관계가 약함

### 상관계수 해석 기준

| 절대값 | 해석 |
|--------|------|
| 0.0 ~ 0.2 | 매우 약한 상관 |
| 0.2 ~ 0.4 | 약한 상관 |
| 0.4 ~ 0.6 | 중간 상관 |
| 0.6 ~ 0.8 | 강한 상관 |
| 0.8 ~ 1.0 | 매우 강한 상관 |

### Pearson vs Spearman 상관계수

- **Pearson**: 선형 관계 측정, 정규분포 가정
- **Spearman**: 순위 기반, 비선형 관계도 감지

In [16]:
# 예시: 광고비와 매출의 상관관계
ad_spend = [100, 150, 200, 250, 300, 350, 400, 450, 500, 550]  # 광고비 (만원)
revenue = [1200, 1450, 1800, 2100, 2400, 2650, 2900, 3200, 3500, 3750]  # 매출 (만원)

# Pearson 상관계수
pearson_corr, pearson_p = stats.pearsonr(ad_spend, revenue)

# Spearman 상관계수
spearman_corr, spearman_p = stats.spearmanr(ad_spend, revenue)

print("광고비-매출 상관분석")
print("=" * 50)
print(f"Pearson 상관계수: {pearson_corr:.4f} (p-value: {pearson_p:.6f})")
print(f"Spearman 상관계수: {spearman_corr:.4f} (p-value: {spearman_p:.6f})")

# 해석
if abs(pearson_corr) > 0.8:
    strength = "매우 강한"
elif abs(pearson_corr) > 0.6:
    strength = "강한"
elif abs(pearson_corr) > 0.4:
    strength = "중간"
elif abs(pearson_corr) > 0.2:
    strength = "약한"
else:
    strength = "매우 약한"

direction = "양의" if pearson_corr > 0 else "음의"

print(f"\n해석: {strength} {direction} 상관관계")

if pearson_p < 0.05:
    print("✅ 통계적으로 유의함")
else:
    print("❌ 통계적으로 유의하지 않음")

광고비-매출 상관분석
Pearson 상관계수: 0.9994 (p-value: 0.000000)
Spearman 상관계수: 1.0000 (p-value: 0.000000)

해석: 매우 강한 양의 상관관계
✅ 통계적으로 유의함


### 💡 Plotly로 산점도 시각화 (px.scatter)

In [17]:
# 산점도로 상관관계 시각화
df_corr = pd.DataFrame({
    '광고비(만원)': ad_spend,
    '매출(만원)': revenue
})

fig = px.scatter(
    df_corr, 
    x='광고비(만원)', 
    y='매출(만원)',
    title=f'광고비 vs 매출 상관관계 (r = {pearson_corr:.3f})',
    trendline='ols',  # 선형 회귀선 추가
    labels={'광고비(만원)': '광고비 (만원)', '매출(만원)': '매출 (만원)'},
    width=700,
    height=500
)

fig.update_traces(
    marker=dict(size=10, color='royalblue'),
    line=dict(color='red', width=2)
)

fig.update_layout(
    font=dict(size=12),
    title_font_size=16,
    showlegend=True
)

fig.show()

### ⚠️ 상관관계 vs 인과관계

**매우 중요한 주의사항**:

- **상관관계 ≠ 인과관계**
- 두 변수가 함께 변한다고 해서 한 변수가 다른 변수를 "일으킨다"고 말할 수 없습니다!

### 잘못된 인과관계 추론 예시

1. **아이스크림 판매량 vs 익사 사고**: 강한 양의 상관 → 원인은 "여름 날씨"
2. **소방차 출동 vs 화재 피해**: 양의 상관 → 소방차가 피해를 일으키지 않음
3. **신발 사이즈 vs 수학 점수**: 어린이 대상 → 원인은 "나이"

In [18]:
# 💡 실무 예시: 다중 변수 상관분석

# 온라인 쇼핑몰 데이터
data = pd.DataFrame({
    '방문자수': [1200, 1450, 1800, 2100, 2400, 2650, 2900, 3200, 3500, 3750],
    '광고비': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550],
    '평균체류시간': [3.2, 3.5, 3.8, 4.1, 4.3, 4.5, 4.7, 4.9, 5.1, 5.3],
    '전환율': [2.1, 2.3, 2.5, 2.7, 2.9, 3.1, 3.3, 3.5, 3.7, 3.9],
    '매출': [120, 145, 180, 210, 240, 265, 290, 320, 350, 375]
})

# 상관계수 행렬
corr_matrix = data.corr()

print("상관계수 행렬:")
print(corr_matrix.round(3))

# 매출과의 상관계수만 추출
print("\n매출과의 상관관계:")
revenue_corr = corr_matrix['매출'].drop('매출').sort_values(ascending=False)
for var, corr in revenue_corr.items():
    print(f"  {var}: {corr:.3f}")

상관계수 행렬:
         방문자수    광고비  평균체류시간    전환율     매출
방문자수    1.000  0.999   0.997  0.999  1.000
광고비     0.999  1.000   0.995  1.000  0.999
평균체류시간  0.997  0.995   1.000  0.995  0.997
전환율     0.999  1.000   0.995  1.000  0.999
매출      1.000  0.999   0.997  0.999  1.000

매출과의 상관관계:
  방문자수: 1.000
  전환율: 0.999
  광고비: 0.999
  평균체류시간: 0.997


### Plotly로 상관계수 히트맵 시각화

In [19]:
# 상관계수 히트맵
fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    aspect='auto',
    color_continuous_scale='RdBu_r',
    title='변수 간 상관계수 히트맵',
    width=600,
    height=600
)

fig.update_layout(
    font=dict(size=11),
    title_font_size=16
)

fig.show()

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. p-value 해석 ⭐

**문제**: 다음 p-value 중 귀무가설을 기각할 수 있는 것을 모두 찾으세요. (유의수준 α = 0.05)

```python
p_values = [0.001, 0.03, 0.08, 0.15, 0.50]
```

**기대 결과**: 귀무가설을 기각할 수 있는 p-value 리스트

In [20]:
p_values = [0.001, 0.03, 0.08, 0.15, 0.50]
# 여기에 코드를 작성하세요


### Q2. 단일 표본 t-test ⭐⭐

**문제**: 한 매장의 일일 평균 매출이 150만원인지 검증하세요. (유의수준 0.05)

```python
daily_sales = [145, 152, 148, 155, 150, 147, 153, 149, 151, 146]
target_avg = 150
```

**출력**: t-statistic, p-value, 결론 (귀무가설 기각 여부)

In [21]:
daily_sales = [145, 152, 148, 155, 150, 147, 153, 149, 151, 146]
target_avg = 150
# 여기에 코드를 작성하세요


### Q3. 독립 표본 t-test ⭐⭐

**문제**: 평일과 주말의 평균 방문자 수에 차이가 있는지 검증하세요.

```python
weekday_visitors = [320, 345, 330, 355, 340, 325, 350]
weekend_visitors = [450, 480, 465, 495, 470, 485, 475]
```

In [22]:
weekday_visitors = [320, 345, 330, 355, 340, 325, 350]
weekend_visitors = [450, 480, 465, 495, 470, 485, 475]
# 여기에 코드를 작성하세요


### Q4. 대응 표본 t-test ⭐⭐⭐

**문제**: 리뉴얼 전후 사이트 이용시간에 유의한 차이가 있는지 검증하세요.

```python
before_renewal = [3.2, 3.5, 3.1, 3.8, 3.4, 3.6, 3.3, 3.7]
after_renewal = [4.1, 4.3, 4.0, 4.5, 4.2, 4.4, 4.1, 4.6]
```

**출력**: 평균 증가량, t-statistic, p-value, 결론

In [23]:
before_renewal = [3.2, 3.5, 3.1, 3.8, 3.4, 3.6, 3.3, 3.7]
after_renewal = [4.1, 4.3, 4.0, 4.5, 4.2, 4.4, 4.1, 4.6]
# 여기에 코드를 작성하세요


### Q5. Chi-square 검정 ⭐⭐⭐

**문제**: 연령대와 제품 구매 여부가 독립적인지 검증하세요.

```python
# 교차표: 행(연령대), 열(구매 여부)
age_purchase = np.array([
    [30, 70],  # 20대: 구매O, 구매X
    [50, 50],  # 30대
    [40, 60]   # 40대
])
```

In [24]:
age_purchase = np.array([
    [30, 70],  # 20대: 구매O, 구매X
    [50, 50],  # 30대
    [40, 60]   # 40대
])
# 여기에 코드를 작성하세요


---

## Part 2 심화 퀴즈

---

### Q6. ANOVA 분석 ⭐⭐⭐

**문제**: 3개 마케팅 채널의 평균 전환율에 차이가 있는지 ANOVA로 검증하세요.

```python
email = [3.2, 3.5, 3.8, 3.1, 3.6]
sns = [4.1, 4.3, 4.5, 3.9, 4.2]
search = [3.0, 2.8, 3.2, 2.9, 3.1]
```

**출력**: 각 채널 평균, F-statistic, p-value, 최고 성과 채널

In [25]:
email = [3.2, 3.5, 3.8, 3.1, 3.6]
sns = [4.1, 4.3, 4.5, 3.9, 4.2]
search = [3.0, 2.8, 3.2, 2.9, 3.1]
# 여기에 코드를 작성하세요


### Q7. Pearson 상관계수 ⭐⭐⭐

**문제**: 공부시간과 시험점수의 상관계수를 계산하고 해석하세요.

```python
study_hours = [2, 3, 5, 4, 6, 7, 8, 5, 6, 7]
test_scores = [60, 65, 75, 70, 80, 85, 90, 72, 78, 88]
```

**출력**: 상관계수, p-value, 상관 강도 해석

In [26]:
study_hours = [2, 3, 5, 4, 6, 7, 8, 5, 6, 7]
test_scores = [60, 65, 75, 70, 80, 85, 90, 72, 78, 88]
# 여기에 코드를 작성하세요


### Q8. 산점도 시각화 ⭐⭐⭐⭐

**문제**: 온도와 아이스크림 판매량의 관계를 `px.scatter`로 시각화하고 상관계수를 제목에 표시하세요.

```python
temperature = [18, 22, 25, 28, 30, 32, 35, 27, 29, 31]
icecream_sales = [150, 180, 220, 250, 280, 310, 350, 240, 270, 300]
```

**요구사항**: trendline 추가, 상관계수를 제목에 포함

In [27]:
temperature = [18, 22, 25, 28, 30, 32, 35, 27, 29, 31]
icecream_sales = [150, 180, 220, 250, 280, 310, 350, 240, 270, 300]
# 여기에 코드를 작성하세요


### Q9. 상관계수 행렬 ⭐⭐⭐⭐

**문제**: 다음 데이터로 상관계수 행렬을 만들고 '매출'과 가장 강한 상관이 있는 변수를 찾으세요.

```python
data = pd.DataFrame({
    '광고비': [100, 150, 200, 250, 300],
    '방문자': [1200, 1800, 2400, 3000, 3600],
    '체류시간': [3.2, 3.8, 4.5, 5.1, 5.8],
    '매출': [120, 180, 240, 300, 360]
})
```

In [28]:
data = pd.DataFrame({
    '광고비': [100, 150, 200, 250, 300],
    '방문자': [1200, 1800, 2400, 3000, 3600],
    '체류시간': [3.2, 3.8, 4.5, 5.1, 5.8],
    '매출': [120, 180, 240, 300, 360]
})
# 여기에 코드를 작성하세요


### Q10. 종합 분석 프로젝트 ⭐⭐⭐⭐⭐

**문제**: 다음 시나리오를 완전히 분석하세요.

**시나리오**: 3개 지점(강남, 홍대, 신촌)의 주말 매출 데이터를 분석하여:
1. ANOVA로 지점 간 차이 검증
2. 각 지점별 평균과 표준편차 계산
3. 유의한 차이가 있다면 최고/최저 성과 지점 파악
4. 상관분석: 방문자 수와 매출의 상관관계 분석

```python
gangnam_sales = [450, 480, 465, 495, 470, 485, 475]
hongdae_sales = [420, 435, 425, 445, 430, 440, 428]
sinchon_sales = [380, 395, 385, 405, 390, 400, 388]

# 방문자 수 (전체 합산)
total_visitors = [3200, 3400, 3300, 3600, 3350, 3500, 3380]
total_sales = [1250, 1310, 1275, 1345, 1290, 1325, 1291]
```

**출력**: 완전한 분석 리포트 (통계량, 검정 결과, 시각화, 비즈니스 인사이트)

In [29]:
gangnam_sales = [450, 480, 465, 495, 470, 485, 475]
hongdae_sales = [420, 435, 425, 445, 430, 440, 428]
sinchon_sales = [380, 395, 385, 405, 390, 400, 388]

total_visitors = [3200, 3400, 3300, 3600, 3350, 3500, 3380]
total_sales = [1250, 1310, 1275, 1345, 1290, 1325, 1291]

# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 개념 핵심 요약

| 개념 | 정의 | 핵심 포인트 |
|------|------|-------------|
| 귀무가설(H₀) | 차이/효과가 없다는 가정 | 기각하려는 대상 |
| 대립가설(H₁) | 차이/효과가 있다는 주장 | 증명하려는 대상 |
| p-value | H₀가 참일 때 현재 데이터가 나올 확률 | p < α → H₀ 기각 |
| 유의수준(α) | H₀ 기각 기준 (보통 0.05) | 엄격할수록 낮게 설정 |
| Type I 오류 | 실제 차이 없는데 "있다" 판단 | False Positive |
| Type II 오류 | 실제 차이 있는데 "없다" 판단 | False Negative |

### Part 2: 검정 방법 핵심 요약

| 검정 방법 | 용도 | Python 함수 | 언제 쓰나? |
|----------|------|-------------|----------|
| 단일 표본 t-test | 1개 그룹 평균 vs 기준값 | `stats.ttest_1samp()` | 우리 회사 vs 업계 평균 |
| 독립 표본 t-test | 2개 독립 그룹 평균 비교 | `stats.ttest_ind()` | A/B 테스트 |
| 대응 표본 t-test | 같은 대상 전후 비교 | `stats.ttest_rel()` | 리뉴얼 전후, 치료 전후 |
| Chi-square | 범주형 변수 독립성 검정 | `stats.chi2_contingency()` | 성별 vs 선호도 |
| ANOVA | 3개 이상 그룹 평균 비교 | `stats.f_oneway()` | 지역별 매출 비교 |
| Pearson 상관 | 선형 상관관계 측정 | `stats.pearsonr()` | 광고비 vs 매출 |
| Spearman 상관 | 비선형 상관관계 측정 | `stats.spearmanr()` | 순위 기반 상관 |

### 💡 실무 팁

1. **p-value 해석 주의**: p < 0.05만 맹신하지 말고 실무적 의미(effect size)도 고려
2. **t-test 선택**: 독립 vs 대응 구분 중요! 같은 사람 전후 = 대응, 다른 사람 = 독립
3. **ANOVA 후 사후분석**: ANOVA에서 유의하면 어떤 그룹이 다른지 추가 분석 필요
4. **상관 ≠ 인과**: 상관관계만으로 인과관계 주장 금지! (아이스크림-익사 예시)
5. **표본 크기**: 너무 작으면 유의한 차이 못 찾고, 너무 크면 사소한 차이도 유의하게 나옴
6. **시각화 필수**: 통계량만 보지 말고 `px.scatter`로 데이터 분포 확인
7. **비즈니스 컨텍스트**: 통계적 유의성과 실무적 중요성은 다를 수 있음

### 📈 가설 검정 의사결정 플로우

```
데이터 유형 확인
    ↓
연속형? → 평균 비교 → 그룹 수?
    1개 → 단일 표본 t-test
    2개 → 독립/대응? → 독립: ttest_ind, 대응: ttest_rel
    3개+ → ANOVA
    
범주형? → 독립성 검정 → Chi-square

연속형 2개? → 상관관계 → Pearson/Spearman
```